In [ ]:
# !python -m pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [3]:
driver = webdriver.Chrome()

driver.get("https://www.investing.com/")
# print(driver.title)

In [4]:
def print_all_elements_by_tag(tag_name: str):
    elem = driver.find_elements(By.TAG_NAME, tag_name)

    for e in elem:
        text = e.text
        if text:
            print(text)

In [5]:
def get_elements_by_tag(driver_, parent_element, tag_type: str):
    elements = []
    
    # Encontre todos os elementos filhos do tipo especificado dentro do elemento pai
    child_elements = parent_element.find_elements(By.TAG_NAME, tag_type)
    
    for element in child_elements:
        # Capture o conteúdo (texto) do elemento
        element_content = element.text
        
        # Verifique se o elemento atual tem filhos do mesmo tipo
        child_structure = get_elements_by_tag(driver_, element, tag_type)
        
        # Se houver filhos, adicione-os como sublista; caso contrário, adicione apenas o conteúdo
        if child_structure:
            elements.append([element_content, child_structure])
        else:
            elements.append(element_content)
    
    return elements

In [6]:
def get_elements_by_incremental_css_name(css_selector_template: str, start: int, end: int):
    # Array para armazenar os textos dos elementos
    elements_texts = []

    # Iterar sobre o intervalo de números (start até end)
    for i in range(start, end + 1):
        # Criar o seletor CSS com o número atual
        css_selector = css_selector_template.format(i)
        try:
            # Tentar encontrar o elemento
            element = driver.find_element(By.CSS_SELECTOR, css_selector)
            # Adicionar o texto do elemento ao array
            elements_texts.append(element.text)
        except:
            # Se o elemento não for encontrado, ignorar e continuar
            print(f"Elemento {css_selector} não encontrado.")
    
    return elements_texts

In [9]:
element = driver.find_element(By.CSS_SELECTOR, '[data-test="homepage-news-main-item-content"]')
element.text

"Israel pounds Beirut and Gaza after rockets hit Israel's north\nBy Maayan Lubell, Maya Gebeily and Nidal al-Mughrabi JERUSALEM/BEIRUT/CAIRO (Reuters) -Israel struck what it said were Hezbollah arms facilities in southern Beirut on Saturday after the Lebanese armed group fired rockets into northern Israel and a spokesman said a drone was launched at Prime Minist..."

In [10]:
elements_texts = get_elements_by_incremental_css_name('[data-test="homepage-news-list-item-{}"]', 1, 3)
elements_texts


['Boeing workers to vote on new wage deal that could end strike',
 'Cuba making slow progress re-starting power after second grid collapse',
 '']